<a href="https://colab.research.google.com/github/adikal25/Automated-Clinical-Documentation-from-Patient-Encounters/blob/main/patient_doctor_notes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from openai import OpenAI
import os
from google.colab import userdata
import base64

api_key = userdata.get("OR_A_TOKEN")

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=api_key,
)

patient_audio_path = "/content/100301_1o00.wav"
doctor_audio_path = "/content/100301_2o00.wav"

# Read binary audio and encode in Base64
with open(patient_audio_path, "rb") as f:
    patient_audio_bytes = f.read()
    patient_audio_base64 = base64.b64encode(patient_audio_bytes).decode('utf-8')

with open(doctor_audio_path, "rb") as f:
    doctor_audio_bytes = f.read()
    doctor_audio_base64 = base64.b64encode(doctor_audio_bytes).decode('utf-8')


PROMPT = """
You are a medical note generator.
Generate a detailed and structured SOAP note based on the provided medical conversation transcript. Organize the information into these clearly labeled sections:
Subjective (S): Summarize all patient-reported symptoms, chief complaints, relevant medical history, and subjective experiences described during the encounter. Ensure this section accurately reflects only the patient's perspective as conveyed in the transcript.
Objective (O): Include all measurable and observable clinical data explicitly mentioned in the transcript, such as vital signs, physical examination findings, laboratory results, and imaging reports. If specific objective data is not documented in the transcript, state 'Not documented in transcript' for that data point or section.
Assessment (A): Provide the clinical diagnosis or differential diagnoses discussed or indicated based on the subjective and objective information extracted solely from the transcript.
Plan (P): Outline the recommended treatment plan, including medications prescribed, therapies suggested, referrals made, diagnostic tests ordered, patient education provided, and follow-up instructions, as discussed within the transcript.

Use professional, concise medical language suitable for clinical documentation. Exclude any irrelevant, redundant, or external information not present in the transcript. Maintain patient confidentiality in the note by using de-identified information and focusing strictly on the medical content of the conversation.
"""

completion = client.chat.completions.create(
    model="openai/gpt-4o-audio-preview",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": PROMPT},
                {"type": "input_audio", "input_audio": {"data": patient_audio_base64, "format": "wav"}},
                {"type": "input_audio", "input_audio": {"data": doctor_audio_base64, "format": "wav"}}
            ]
        }
    ]
)

print(completion.choices[0].message.content)

SOAP Note:

Subjective (S):
- Chief Complaint: Patient reports experiencing asthma attacks and "pins and needles" sensations in the chest, arms, and hands.
- History of Present Illness: The patient describes current symptoms of chest congestion, persistent coughing, and unusual sensations when coughing. The patient mentions that their family members have also been ill with chest congestion and frequent coughing. The patient expresses concern regarding whether they should simply stay comfortable and endure the sensations or seek medical evaluation.
- Relevant Past Medical History: The patient reports having had two heart stents placed within the past 11 months. The patient noted that they have not experienced asthma attacks for a few years until now.
- Social Context: The patient ended the conversation prematurely due to concerns about associated fees.

Objective (O):
- Not documented in transcript. No vital signs, physical exam findings, or diagnostic test results provided.

Assessment